In [317]:
# importing necessery libraries
import pandas as pd
import numpy as np
import csv
import re
# import socket as sc
from urllib.parse import urlparse
from tld import get_tld
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
# from IPy import IP

In [330]:
file = "urls_all.csv"
df = pd.read_csv(file, sep="\t", low_memory=False)

In [331]:
# for getting the same urls' format, we delete http:// and https://
df["url"] = df["url"].str.replace("http://", '')
df["url"] = df["url"].str.replace("https://", '')

df["label"] = [int(x) for x in df["url_type"]]

# cleaning last dash ("/") if there are some
df["url"] = [x.rstrip('/') for x in df["url"]]
df["url"] = [x.lstrip('www.') for x in df["url"]]

In [349]:
del df_tmp
# for processing faster, we temporaty will work with only 10000 rows
df_tmp=df[["url", "ip", "label"]]

In [350]:
# function counts % of vowels
def vowels_rate(ip_str):
    v_dic = 'AEIOUYaeiouy'
    c_dic = 'BCDFGHJKLMNPQRSTVWXZbcdfghjklmnpqrstvwxz'

    v_count = {}.fromkeys(v_dic,0)
    for char in ip_str:
       if char in v_count:
           v_count[char] += 1
            
    c_count = {}.fromkeys(c_dic,0)
    for char in ip_str:
       if char in c_count:
           c_count[char] += 1

    v = sum(v_count.values()) # vowels
    c = sum(c_count.values()) # consonants
    s = v+c # sum of vowels and consonants
    
    if s==0:
        rate=0
    else:
        rate = v/s
    
    return round(rate, 4) 
    
# function that extracts authority part of a URL
def get_tdl(raw_url):
    urlnetloc=[]
    for i in raw_url:
        url = "//"+i
        o=urlparse(url)
        urlnetloc.append(o.netloc)
    return urlnetloc

In [351]:
# Extracting lexicon based features
df_tmp["url_length"]=df_tmp["url"].str.len()
df_tmp["tdl"] = get_tdl(df_tmp["url"])
df_tmp["tdl_length"] = df_tmp["tdl"].str.len()
df_tmp["content_length"]=df_tmp["url_length"] - df_tmp["tdl_length"]
df_tmp["special_characters"] = df_tmp['url'].str.count(r'[^A-Za-z0-9]+')
df_tmp["slashs"] = df_tmp["url"].str.count("/")
df_tmp["vowels_rate"] = [vowels_rate(i) for i in df_tmp["url"]]
df_tmp["is_hyphen_tdl"] = [int(elem) for elem in df_tmp["tdl"].str.contains("-")]
df_tmp["is_digit_tdl"] = [int(elem) for elem in df_tmp["tdl"].str.contains(r"[0-9]")]
df_tmp["is_ip_based"] = [int(elem) for elem in df_tmp["tdl"].str.contains(r"\b((?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(?:(?<!\.)\b|\.)){4}")]
df_tmp["is_hex_based"] = [int(elem) for elem in df_tmp["tdl"].str.contains(r"^[0-9A-Fa-f]+$")]
df_tmp["is_underscore"] = [int(elem) for elem in df_tmp["url"].str.contains("_")]
df_tmp["is_equal"] = [int(elem) for elem in df_tmp["url"].str.contains("=")]
df_tmp["is_comma"] = [int(elem) for elem in df_tmp["url"].str.contains(",")]
df_tmp["is_semicolon"] = [int(elem) for elem in df_tmp["url"].str.contains(";")]
df_tmp["is_query_part"] = [int(elem) for elem in df_tmp["url"].str.contains(r"[\\?&]([^&=]+)=([^&=]+)")]
df_tmp["is_aspersed"] = [int(elem) for elem in df_tmp["url"].str.contains("@")]
df_tmp["is_username"] = [int(elem) for elem in df_tmp["url"].str.contains("username")]
df_tmp["is_password"] = [int(elem) for elem in df_tmp["url"].str.contains("password")]
df_tmp["is_nonstandard_port"] = [int(elem) for elem in df_tmp["url"].str.contains(r"\:\d{4}")]
# df_tmp["is_hyphen_url"] = df_tmp["url"].str.contains("-")
# df_tmp["is_parameter_part"] = [int(elem) for elem in df_tmp["url"].str.contains(r"(\?|\&)([^=]+)\=([^&]+)")]
# df_tmp["is_fragment_part"] = df_tmp["url"].str.contains(r"(#[\w\-]+)?$")

# Delete unnecessary features
del df_tmp["tdl"]
del df_tmp["url"]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
# Extracting host based features
# df_tmp["server"] = df_tmp["url"].str.contains("-")
# df_tmp["whois_country"] = df_tmp["url"].str.contains("-")
# df_tmp["whois_regdate"] = df_tmp["url"].str.contains("-")
# df_tmp["whois_updated_date"] = df_tmp["url"].str.contains("-")

In [354]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4218916 entries, 0 to 4218915
Data columns (total 21 columns):
ip                     object
label                  int64
url_length             int64
tdl_length             int64
content_length         int64
special_characters     int64
slashs                 int64
vowels_rate            float64
is_hyphen_tdl          int64
is_digit_tdl           int64
is_ip_based            int64
is_hex_based           int64
is_underscore          int64
is_equal               int64
is_comma               int64
is_semicolon           int64
is_query_part          int64
is_aspersed            int64
is_username            int64
is_password            int64
is_nonstandard_port    int64
dtypes: float64(1), int64(19), object(1)
memory usage: 675.9+ MB


In [355]:
# combining all dataframes into big one and exporting to csv file 
df_tmp.to_csv("urls_final.csv", sep=';')
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4218916 entries, 0 to 4218915
Data columns (total 21 columns):
ip                     object
label                  int64
url_length             int64
tdl_length             int64
content_length         int64
special_characters     int64
slashs                 int64
vowels_rate            float64
is_hyphen_tdl          int64
is_digit_tdl           int64
is_ip_based            int64
is_hex_based           int64
is_underscore          int64
is_equal               int64
is_comma               int64
is_semicolon           int64
is_query_part          int64
is_aspersed            int64
is_username            int64
is_password            int64
is_nonstandard_port    int64
dtypes: float64(1), int64(19), object(1)
memory usage: 675.9+ MB
